In [17]:
using CSV
using DataFrames

In [18]:
shots = DataFrame(CSV.File(raw"C:\Users\liber\Documents\Datasets\Football\shots.csv"))

Row,gameID,shooterID,assisterID,minute,situation,lastAction,shotType,shotResult,xGoal,positionX,positionY
,Int64,Int64,String7,Int64,String15,String15,String15,String15,Float64,Float64,Float64
1,81,554,NA,27,DirectFreekick,Standard,LeftFoot,BlockedShot,0.104347,0.794,0.421
2,81,555,631,27,SetPiece,Pass,RightFoot,BlockedShot,0.0643422,0.86,0.627
3,81,554,629,35,OpenPlay,Pass,LeftFoot,BlockedShot,0.0571568,0.843,0.333
4,81,554,NA,35,OpenPlay,Tackle,LeftFoot,MissedShots,0.0921414,0.848,0.533
5,81,555,654,40,OpenPlay,BallRecovery,RightFoot,BlockedShot,0.035742,0.812,0.707
6,81,555,629,49,OpenPlay,Pass,LeftFoot,MissedShots,0.0212247,0.725,0.545
7,81,631,NA,64,OpenPlay,None,RightFoot,SavedShot,0.103004,0.896,0.597
8,81,629,557,72,OpenPlay,Cross,RightFoot,MissedShots,0.106513,0.89,0.534
9,81,629,NA,76,DirectFreekick,Standard,RightFoot,MissedShots,0.0430677,0.747,0.694


In [19]:
games = DataFrame(CSV.File(raw"C:\Users\liber\Documents\Datasets\Football\games.csv"));

In [20]:
players = DataFrame(CSV.File(raw"C:\Users\liber\Documents\Datasets\Football\players.csv"));

In [21]:
players[players.name .== "Lionel Messi", :]

Row,playerID,name
,Int64,String31
1,2097,Lionel Messi


In [22]:
players[players.name .== "Neymar", :]

Row,playerID,name
,Int64,String31
1,2099,Neymar


In [23]:
players[players.name .== "Karim Benzema", :]

Row,playerID,name
,Int64,String31
1,2370,Karim Benzema


In [24]:
players[players.name .== "Rodrygo", :]

Row,playerID,name
,Int64,String31
1,8015,Rodrygo


In [25]:
players[players.name .== "Harry Kane", :]

Row,playerID,name
,Int64,String31
1,647,Harry Kane


In [26]:
messi = 2097
neymar = 2099
benzema = 2370
rodrygo = 8015
kane = 647
players = [neymar, messi, benzema, rodrygo, kane];

In [27]:
shots_players = [shots[shots.shooterID .== player, :] for player in players];

In [28]:
using FileIO, GLMakie, ImageShow, ColorSchemes

In [29]:
img = load(raw"C:\Users\liber\Documents\Datasets\Football\Football_field_cut.png");

In [30]:
#figura em 4k
f = Figure(resolution = (3840, 2160))
imagem = Axis(f[1, 2], aspect=AxisAspect(100/65))
#remover eixos
hidedecorations!(imagem)
#adicionando imagem de fundo / campo
background = image!(0:1, 0:1, rotr90(img); fxaa=true)

#grid com botoes em baixo 
f[2, :] = buttongrid = GridLayout()

#nome dos botoes
labels = ["Neymar", "Messi", "Benzema", "Rodrygo", "H. Kane"]
buttons = buttongrid[1, 1:5] = [Button(f, label=l) for l in labels]

#nome toggles

#dados
player = Observable(1)
data = @lift(shots_players[$player])

#definir pontos | IMPORTANTE USAR ESTRUTURA "Point2f()"
points = @lift(Point2f.($data.positionX, $data.positionY))
#Xg para o heat map
hmap = @lift($data.xGoal)

#scatter plot
plot = scatter!(imagem, points, color=hmap, colormap=cgrad([:cyan, :blue, :purple, :red, :orange, :yellow], [0.1, 0.2, 0.4, 0.5, 0.7, 0.9], alpha = 0.6, rev = false, scale = nothing), strokewidth=0.8, fxaa=true)

#barra com a legenda do heat map
barra = Colorbar(f[1, 3], plot; label = "xGoals", ticks=0:0.1:1)

#menu 1 : (label, funcao)
feet = ["RightFoot", "LeftFoot", "Head", "Todos"]
data_funcs_1 = [i != "Todos" ? (data -> data[data.shotType .== i, :]) : data -> data for i in feet]
menu1 = Menu(f, options = zip(feet, data_funcs_1), default = "Todos")

#menu 2 : (label, funcao)
outcome = ["Goal", "MissedShots", "SavedShot", "Todos"]
data_funcs_2 = [i != "Todos" ? (data -> data[data.shotResult .== i, :]) : data -> data for i in outcome]
menu2 = Menu(f, options = zip(outcome, data_funcs_2), default = "Todos")

#grid de menus
f[:, 1] = vgrid!(
    Label(f, "Foot", width = nothing),
    menu1,
    Label(f, "Outcome", width = nothing),
    menu2;
    tellheight = false, width = 200)

#configurando ativacao dos botoes | levar em conta os menus ativados
for i in 1:5
    on(buttons[i].clicks) do click
        player[] = i
        data[] = menu1.selection.val(menu2.selection.val(shots_players[player.val]))
    end
end

#configurando so menus | importante manter o menu ja ativado 
on(menu1.selection) do s
    data.val = menu2.selection.val(shots_players[player.val])
    data[] = s(data.val)
end

on(menu2.selection) do s
    data.val = menu1.selection.val(shots_players[player.val])
    data[] = s(data.val)
end

ObserverFunction defined at c:\Users\liber\Documents\GitHub\Portifolio\Futebol_visualizacao.ipynb:66 operating on Observable{Any}(#55)

In [31]:
f